In [1]:
import torch
from datasets import load_from_disk, Dataset, DatasetDict
from tqdm import tqdm
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def logit_stats_gpu(logits, k=10, is_softmax=False):
    # logits: [B, V], torch.tensor on cuda
    probs = torch.softmax(logits, dim=-1) if not is_softmax else logits
    entropy = -(probs * (probs + 1e-12).log()).sum(dim=-1)
    max_prob, _ = probs.max(dim=-1)
    mean = probs.mean(dim=-1)
    std = probs.std(dim=-1)
    topk_probs, _ = probs.topk(k, dim=-1)
    return entropy, max_prob, mean, std, topk_probs

def batch_extract_features(lh, eh, k=10):
    entropy_lh, max_lh, mean_lh, std_lh, topk_lh = logit_stats_gpu(lh, k)
    entropy_eh, max_eh, mean_eh, std_eh, topk_eh = logit_stats_gpu(eh, k)
    features = torch.cat([
        entropy_lh.unsqueeze(1), max_lh.unsqueeze(1), mean_lh.unsqueeze(1), std_lh.unsqueeze(1), topk_lh,
        entropy_eh.unsqueeze(1), max_eh.unsqueeze(1), mean_eh.unsqueeze(1), std_eh.unsqueeze(1), topk_eh
    ], dim=1)
    return features

def process_split(ds, k=10, batch_size=1024, last_logit_col="last_logit", egale_logit_col="egale_1st_forward_logit", accept_length_col="accept_length"):
    ds.set_format(type="torch", columns=[last_logit_col, egale_logit_col, accept_length_col])
    all_features = []
    all_accept_length = []
    for start in tqdm(range(0, len(ds), batch_size)):
        batch = ds[start : start + batch_size]
        lh = batch[last_logit_col].to("cuda")
        eh = batch[egale_logit_col].to("cuda")
        features = batch_extract_features(lh, eh, k=k)
        all_features.append(features.cpu())
        all_accept_length.extend(batch[accept_length_col].tolist())
    all_features = torch.cat(all_features, dim=0)
    new_dataset = Dataset.from_dict({
        "features": all_features.tolist(),
        "accept_length": all_accept_length
    })
    return new_dataset

def main():
    input_dataset_dir = "../data/mt-bench-llama3-d13-topk10-t0"
    output_dir = "../data/mt-bench-llama3-d13-topk10-t0-cal"
    os.makedirs(output_dir, exist_ok=True)
    last_logit_col = "last_logit"
    egale_logit_col = "egale_1st_forward_logit"
    accept_length_col = "accept_length"
    k = 10
    batch_size = 1024

    dataset_dict = load_from_disk(input_dataset_dir)
    for split, ds in dataset_dict.items():
        print(f"Processing split: {split}")
        new_ds = process_split(ds, k=k, batch_size=batch_size, last_logit_col=last_logit_col, egale_logit_col=egale_logit_col, accept_length_col=accept_length_col)
        out_json = os.path.join(output_dir, f"dataset_{split}.json")
        new_ds.to_json(out_json)
        print(f"Saved {split} split to {out_json}")

In [ ]:
main()

Processing split: train


  0%|          | 0/7 [05:50<?, ?it/s]


KeyboardInterrupt: 

: 

In [6]:
def main1():
    input_dataset_dir = "../data/mt-bench-llama3-d13-topk10-t0"
    output_dir = "../data/mt-bench-llama3-d13-topk10-t0-cal"
    os.makedirs(output_dir, exist_ok=True)
    last_logit_col = "last_logit"
    egale_logit_col = "egale_1st_forward_logit"
    accept_length_col = "accept_length"
    k = 10
    batch_size = 1024

    dataset_dict = load_from_disk(input_dataset_dir)
    processed_splits = {}  # 保存所有 split 的新数据集

    for split, ds in dataset_dict.items():
        print(f"Processing split: {split}")
        new_ds = process_split(
            ds, k=k, batch_size=batch_size,
            last_logit_col=last_logit_col,
            egale_logit_col=egale_logit_col,
            accept_length_col=accept_length_col
        )
        processed_splits[split] = new_ds
        # 如需单独保存json，可加下面两行：
        # out_json = os.path.join(output_dir, f"dataset_{split}.json")
        # new_ds.to_json(out_json)

    # 保存为Arrow格式（DatasetDict），可直接用load_from_disk读取
    output_arrow_dir = os.path.join(output_dir, "arrow")
    DatasetDict(processed_splits).save_to_disk(output_arrow_dir)
    print(f"All splits saved to: {output_arrow_dir}")

In [7]:
main1()

Processing split: train


100%|██████████| 7/7 [00:02<00:00,  2.54it/s]


Processing split: test


Saving the dataset (1/1 shards): 100%|██████████| 1601/1601 [00:00<00:00, 287091.95 examples/s]

All splits saved to: ../data/mt-bench-llama3-d13-topk10-t0-cal/arrow


In [4]:
from datasets import load_from_disk, concatenate_datasets
dataset1 = load_from_disk("../data/scores_rb/shareGPT-vicuna-d7-topk10-t1-0")
dataset2 = load_from_disk("../data/scores_rb/shareGPT-vicuna-d7-topk10-t1-1")

In [5]:
dataset = concatenate_datasets([dataset1, dataset2])
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset.save_to_disk("../data/scores_rb/shareGPT-vicuna-d7-topk10-t1")

Saving the dataset (0/1 shards):   0%|          | 0/107284 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/26821 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['eagle_1_forward', 'eagle_2_forward', 'eagle_3_forward', 'eagle_4_forward', 'eagle_5_forward', 'eagle_6_forward', 'eagle_7_forward', 'eagle_8_forward', 'action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7'],
        num_rows: 107284
    })
    test: Dataset({
        features: ['eagle_1_forward', 'eagle_2_forward', 'eagle_3_forward', 'eagle_4_forward', 'eagle_5_forward', 'eagle_6_forward', 'eagle_7_forward', 'eagle_8_forward', 'action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7'],
        num_rows: 26821
    })
})

In [21]:
[x["stop"] for x in [dataset["train"][9][f"action_{i}"] for i in range(8)]]

[[0.00038242340087890625, 0.9998638033866882],
 [0.0009708404541015625, 0.035552978515625, 0.9637311100959778],
 [0.0009708404541015625, 0.04150390625, 0.014312744140625, 0.9433665871620178],
 [0.0009708404541015625,
  0.04486083984375,
  0.024749755859375,
  0.1595458984375,
  0.7700685262680054],
 [0.0009746551513671875,
  0.04656982421875,
  0.024200439453125,
  0.1602783203125,
  0.040496826171875,
  0.7277655601501465],
 [0.0016307830810546875,
  0.045928955078125,
  0.178955078125,
  0.018096923828125,
  0.0296630859375,
  0.0992431640625,
  0.6266781091690063],
 [0.0019741058349609375,
  0.04571533203125,
  0.1861572265625,
  0.01043701171875,
  0.0301971435546875,
  0.098388671875,
  0.038299560546875,
  0.5888843536376953],
 [0.0021820068359375,
  0.056121826171875,
  0.175537109375,
  0.01158905029296875,
  0.034912109375,
  0.0941162109375,
  0.041717529296875,
  0.038360595703125,
  0.5456371307373047]]

In [6]:
import datasets

dataset = datasets.load_from_disk("../data/scores_rb/shareGPT-ds-d7-topk10-t1-0")
dataset

DatasetDict({
    train: Dataset({
        features: ['eagle_1_forward', 'eagle_2_forward', 'eagle_3_forward', 'eagle_4_forward', 'eagle_5_forward', 'eagle_6_forward', 'eagle_7_forward', 'eagle_8_forward', 'action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7'],
        num_rows: 69463
    })
    test: Dataset({
        features: ['eagle_1_forward', 'eagle_2_forward', 'eagle_3_forward', 'eagle_4_forward', 'eagle_5_forward', 'eagle_6_forward', 'eagle_7_forward', 'eagle_8_forward', 'action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7'],
        num_rows: 17366
    })
})

In [5]:
import datasets

dataset = datasets.load_from_disk("../data/scores_rb/shareGPT-vicuna-d7-topk10-t1")
dataset

DatasetDict({
    train: Dataset({
        features: ['eagle_1_forward', 'eagle_2_forward', 'eagle_3_forward', 'eagle_4_forward', 'eagle_5_forward', 'eagle_6_forward', 'eagle_7_forward', 'eagle_8_forward', 'action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7'],
        num_rows: 107284
    })
    test: Dataset({
        features: ['eagle_1_forward', 'eagle_2_forward', 'eagle_3_forward', 'eagle_4_forward', 'eagle_5_forward', 'eagle_6_forward', 'eagle_7_forward', 'eagle_8_forward', 'action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7'],
        num_rows: 26821
    })
})